<a href="https://colab.research.google.com/github/plaban1981/Langchain_usecases/blob/main/Custom_medical_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qU langchain openai tiktoken duckduckgo-search

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 56.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 54.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import langchain
from langchain.prompts.base import StringPromptTemplate
from langchain.prompts import PromptTemplate,StringPromptTemplate
from langchain.agents import Tool, AgentExecutor, AgentOutputParser,LLMSingleActionAgent,initialize_agent
from langchain import OpenAI,LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.tools import DuckDuckGoSearchRun
from langchain.schema import AgentAction,AgentFinish
import re
from typing import List,Union
import os
from getpass import getpass

In [3]:
os.environ["OPENAI_API_KEY"] = getpass()

··········


In [4]:
search = DuckDuckGoSearchRun()
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    )
]

In [5]:
obj = search.run("How can I treat a sprained ankle?")
print(type(obj))
print(obj)

<class 'str'>
Sprained ankle — Injury to a ligament of the ankle can usually be treated with at-home care and appropriate exercises to get you back on your feet. This content does not have an English version. This content does not have an Arabic version. Skip to content Care at Mayo Clinic Care at Mayo Clinic About Mayo Clinic Request Appointment Find a Doctor Sprains - Diagnosis and treatment - Mayo Clinic A sprain is a stretching or tearing of ligaments — the tough bands of tissue that connect two bones together in your joints. Ice and elevation can help. This content does not have an English version. This content does not have an Arabic version. Skip to content Care at Mayo Clinic A sprain is a stretching or tearing of ligaments — the tough bands of fibrous tissue that connect two bones together in your joints. The most common location for a sprain is in your ankle. Initial treatment includes rest, ice, compression and elevation. Mild sprains can be successfully treated at home. Her

In [6]:
search.run("site:webmd.com How can I treat a sprained ankle?")

"RICE. 2 /14. It stands for rest, ice, compress, and elevate, and it's the go-to treatment for minor sprains and bruising. Ice the injury as soon as possible and put pressure on it with an ... Seek medical help for burns if they are severe; on the face, hands, feet, or genitals; or bigger than 2 inches. For treating small minor burns at home, rinse the area under running cool water ... You treat it the same way: Rest it, ice it, raise it, and use pain relievers. When to Call the Doctor . 8/13 . A bruise can need medical care if: You think a sprain or broken bone may have caused it. Treatment of myositis varies according to the cause. Causes of Myositis. Myositis is caused by any condition that leads to inflammation in the muscles. Myositis causes can be divided into several ... Being overweight. Stress fractures in your toes or metatarsals. Bunions. Torn ligaments in your feet. A condition known as Morton's neuroma (interdigital neuroma) also causes metatarsalgia-like ..."

In [7]:
def duck_wrapper(input_text):
    search_results = search.run(f"site:webmd.com {input_text}")
    return search_results

tools = [
    Tool(
        name = "Search WebMD",
        func=duck_wrapper,
        description="useful for when you need to answer medical and pharmalogical questions"
    )
]

## Prompt Template
This instructs the agent on what to do. Generally, the template should incorporate:

**`tools:`** which tools the agent has access and how and when to call them.

**`intermediate_steps:`** These are tuples of previous (**`AgentAction`**, **`Observation`**) pairs. These are generally not passed directly to the model, but the prompt template formats them in a specific way.

**`input:`** generic user input

In [8]:
# Set up the base template
template = """Answer the following questions as best you can, but speaking as compasionate medical professional. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Remember to answer as a compansionate medical professional when giving your final answer.

Previous conversation history:
{history}

Question: {input}
"""

In [9]:
tool_names = [tool.name for tool in tools]
history = None
prompt = PromptTemplate(template=template,input_variables=["tools","tool_names","history","input"])

In [10]:
prompt

PromptTemplate(input_variables=['tools', 'tool_names', 'history', 'input'], output_parser=None, partial_variables={}, template='Answer the following questions as best you can, but speaking as compasionate medical professional. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin! Remember to answer as a compansionate medical professional when giving your final answer.\n\nPrevious conversation history:\n{history}\n\nQuestion: {input}\n', template_format='f-string', validate_template=True)

In [11]:
llm= OpenAI()

In [74]:
from langchain.agents.agent_types import AgentType
agent = initialize_agent(tools=tools,
                         llm=ChatOpenAI(),
                         agent = AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
                         verbose=False,
                         return_intermediate_steps=True
                         )

In [80]:
try:
    response = agent({"input":"how to treat acid reflux?",
                      "tools":tools,
                      "tool_names":tool_names,
                      "history":None,
                      "chat_history":None},
                     return_only_outputs=True)
except ValueError as e:
    response = str(e)
    if not response.startswith("Could not parse LLM output: `"):
        raise e
    response = response.removeprefix("Could not parse LLM output: `").removesuffix("`")

In [81]:
print(response)

{'output': 'There are a few lifestyle changes and home remedies that can help treat acid reflux, such as avoiding food triggers, eating smaller and more frequent meals, elevating the head of the bed while sleeping, and avoiding lying down after eating. Over-the-counter medications such as antacids and proton pump inhibitors can also be effective in treating acid reflux. However, it is always best to consult with a doctor if symptoms persist or worsen.', 'intermediate_steps': [(AgentAction(tool='Search WebMD', tool_input='how to treat acid reflux', log='Thought: Do I need to use a tool? Yes\nAction: Search WebMD\nAction Input: how to treat acid reflux'), 'Eat Earlier. Going to bed on a full stomach makes nighttime heartburn more likely. A full stomach puts pressure on the valve at the top of the stomach, which is supposed to keep stomach acid out ... Trim the fat off of meat and poultry, and cut the skin off chicken. Tweaks like these might be enough to tame your heartburn. Tomatoes (in

In [86]:
try:
    response = agent({"input":"Any specific medications I need to take ?","tools":tools,"tool_names":tool_names,"history":response['output'],"chat_history":response['output']})
except ValueError as e:
    response = str(e)
    if not response.startswith("Could not parse LLM output: `"):
        raise e
    response = response.removeprefix("Could not parse LLM output: `").removesuffix("`")

In [87]:
response

{'input': 'Any specific medications I need to take ?',
 'tools': [Tool(name='Search WebMD', description='useful for when you need to answer medical and pharmalogical questions', args_schema=None, return_direct=False, verbose=False, callbacks=None, callback_manager=None, handle_tool_error=False, func=<function duck_wrapper at 0x7fb68ff628c0>, coroutine=None)],
 'tool_names': ['Search WebMD'],
 'history': 'There are several over-the-counter medications that can help relieve acid reflux symptoms, such as antacids, histamine receptor blockers (H2 blockers), and proton pump inhibitors (PPIs). However, it is always best to consult with a doctor before starting any new medication, as they can provide personalized recommendations based on your specific symptoms and medical history.',
 'chat_history': 'There are several over-the-counter medications that can help relieve acid reflux symptoms, such as antacids, histamine receptor blockers (H2 blockers), and proton pump inhibitors (PPIs). However,

In [88]:
response['output']

"The specific medication you need to take for acid reflux will depend on your individual symptoms and medical history. It's best to consult with a doctor or healthcare provider to determine the most appropriate medication for you. They may recommend antacids, H2 blockers, or PPIs, or a combination of these medications. It's important to follow their recommendations and take the medication as directed to achieve the best results."

## Output Parser

In [17]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [18]:
disease_schema = ResponseSchema(name="Ailment",
                                description = "This is the name of the diesease or ailment for which the query has been asked")
#
treatment_schema = ResponseSchema(name="Treatment",
                                  description = " This is the treatment suggested for the ailment or diesease specified")
#
response_schemas = [disease_schema,treatment_schema]


In [19]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()

In [20]:
format_instructions

'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"Ailment": string  // This is the name of the diesease or ailment for which the query has been asked\n\t"Treatment": string  //  This is the treatment suggested for the ailment or diesease specified\n}\n```'

In [38]:
# Set up the base template
template = """Answer the following questions as best you can, but speaking as compasionate medical professional. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Remember to answer as a compansionate medical professional when giving your final answer.

Previous conversation history:
{history}

Question: {input}

Based on the Question asked provide the name of the ailment and treatment. Format the output with a JSON with the following keys:
Ailment
Treatment
{format_instructions}
"""

In [46]:
response = agent.run({"input":"how to treat acid reflux?","tools":tools,"tool_names":tool_names,"format_instructions":format_instructions,"history":None,"chat_history":None})



> Entering new  chain...
Thought: Do I need to use a tool? Yes
Action: Search WebMD
Action Input: how to treat acid reflux
Observation: Eat Earlier. Going to bed on a full stomach makes nighttime heartburn more likely. A full stomach puts pressure on the valve at the top of the stomach, which is supposed to keep stomach acid out ... Esophagitis Symptoms. Symptoms of esophagitis include: Difficult or painful swallowing. Acid reflux. Heartburn. A feeling of something of being stuck in the throat. Chest pain. Nausea. Vomiting. Abdominal bloating. Abdominal pain. Vomiting. Indigestion. Burning or gnawing feeling in the stomach between meals or at night. Hiccups. Loss of appetite. Vomiting blood or coffee ground-like ... Trim the fat off of meat and poultry, and cut the skin off chicken. Tweaks like these might be enough to tame your heartburn. Tomatoes (including foods like salsa and marinara sauce) and citrus ... Intravenous or injected magnesium is used to treat other conditions, ... (

In [47]:
print(response)

There are several ways to treat acid reflux, including making lifestyle changes such as eating earlier and avoiding trigger foods like tomatoes and citrus, as well as taking medication such as proton pump inhibitors (PPIs). It's also important to talk to a doctor if symptoms persist or worsen.


In [48]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.0)

prompt = ChatPromptTemplate(
    messages=[
        HumanMessagePromptTemplate.from_template(template)
    ],
    input_variables=["input","tools","tool_names","history"],
    partial_variables={"format_instructions": format_instructions},
    output_parser=output_parser # here we add the output parser to the Prompt template
)

In [64]:
print(prompt.format(input="How to treat a twisted ankle?",
                    tools=tools,
                    tool_names = tool_names,
                    history=None))

Human: Answer the following questions as best you can, but speaking as compasionate medical professional. You have access to the following tools:

[Tool(name='Search WebMD', description='useful for when you need to answer medical and pharmalogical questions', args_schema=None, return_direct=False, verbose=False, callbacks=None, callback_manager=None, handle_tool_error=False, func=<function duck_wrapper at 0x7fb68ff628c0>, coroutine=None)]

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [['Search WebMD']]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Remember to answer as a compansionate medical professional when giving your final answer.

Previous conversation hist

In [65]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm,
                 prompt=prompt)

In [66]:
response = chain.predict_and_parse(input="How to treat a twisted ankle?",
                    tools=tools,
                    tool_names = tool_names,
                    history=None)

In [67]:
response

{'Ailment': 'Twisted ankle',
 'Treatment': 'Rest, ice, compression, and elevation (RICE). Over-the-counter pain relievers can also be taken to reduce pain and swelling. In severe cases, a cast or brace may be necessary.'}

In [68]:
print(prompt.format(input="How to treat acid reflux?",
                    tools=tools,
                    tool_names = tool_names,
                    history=None))

Human: Answer the following questions as best you can, but speaking as compasionate medical professional. You have access to the following tools:

[Tool(name='Search WebMD', description='useful for when you need to answer medical and pharmalogical questions', args_schema=None, return_direct=False, verbose=False, callbacks=None, callback_manager=None, handle_tool_error=False, func=<function duck_wrapper at 0x7fb68ff628c0>, coroutine=None)]

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [['Search WebMD']]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Remember to answer as a compansionate medical professional when giving your final answer.

Previous conversation hist

In [69]:
response = chain.predict_and_parse(input="How to treat acid reflux?",
                    tools=tools,
                    tool_names = tool_names,
                    history=None,
                    output_parser=output_parser)

In [70]:
response

{'Ailment': 'Acid reflux',
 'Treatment': "Treatment options for acid reflux may include lifestyle changes such as avoiding trigger foods, losing weight, and quitting smoking. Over-the-counter medications like antacids and proton pump inhibitors can also help. In more severe cases, prescription medications or surgery may be necessary. It's important to speak with a healthcare provider before starting any new treatment regimen."}